In [1]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import pickle



In [2]:
# %%
# Load and process the data
df = pd.read_csv('Updated_Common_Data_Morrisons.csv')

# Convert date from 'YYYYMMDD' to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Extract features from date
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.dayofweek

# Drop the original date column
df = df.drop(columns=['date','own_brand'])



In [3]:

# %%
df.head()


,supermarket,unit_price,unit,names,category,ASDA_price,Tesco_price,Sains_price,year,month,day,weekday
0,Morrisons,2.8,l,Monster Energy Zero Sugar,drinks,3.40,3.000000,2.9500,2024,1,29,0
1,Morrisons,4.4,kg,McCain Potato Smiles,food_cupboard,5.51,5.510000,5.5100,2024,1,29,0
2,Morrisons,14.7,kg,Walkers Ready Salted Multipack Crisps,food_cupboard,13.30,11.750000,14.7000,2024,1,29,0
3,Morrisons,14.7,kg,Walkers Cheese & Onion Multipack Crisps,food_cupboard,13.30,10.500000,14.7000,2024,1,29,0
4,Morrisons,2.5,l,Coca-Cola Original Taste,drinks,1.60,3.128571,3.2375,2024,1,29,0


In [4]:
df.duplicated().sum()

0

In [4]:

# Encode categorical features
label_encoders = {}
for column in ['supermarket', 'names','unit','category']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Function to create sequences for LSTM
def create_sequences(group, sequence_length):
    X, y = [], []
    for i in range(len(group) - sequence_length):
        X.append(group.iloc[i:i + sequence_length].drop(columns='unit_price').values)
        y.append(group.iloc[i + sequence_length]['unit_price'])
    return np.array(X), np.array(y)

sequence_length = 10
X_list, y_list = [], []

# Group by the product or other unique identifier column
for _, group in df.groupby('names'):
    X_group, y_group = create_sequences(group, sequence_length)
    X_list.append(X_group)
    y_list.append(y_group)

# Concatenate all groups
X = np.concatenate(X_list)
y = np.concatenate(y_list)

# Ensure correct data types
X = X.astype(np.float32)
y = y.astype(np.float32)

In [5]:


# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure correct data types for train and test sets
train_X = train_X.astype(np.float32)
train_y = train_y.astype(np.float32)
test_X = test_X.astype(np.float32)
test_y = test_y.astype(np.float32)

# Custom RMSE loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Build the enhanced LSTM model with additional layers
model = Sequential()

# First LSTM layer with Dropout
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))

# Second LSTM layer with Dropout
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

# Third LSTM layer with Dropout
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

# Fourth LSTM layer with Dropout
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))

# First Dense layer
model.add(Dense(25, activation='relu'))

# Second Dense layer
model.add(Dense(10, activation='relu'))

# Output layer
model.add(Dense(1))

# Compile the model with the custom RMSE loss function
model.compile(optimizer='adam', loss=rmse)

# Train the model
model.fit(train_X, train_y, epochs=100, batch_size=64, validation_split=0.1)

# Evaluate the model
loss = model.evaluate(test_X, test_y)
print(f"Test RMSE: {loss}")



Epoch 1/100
124/124 [==============================] - 9s 30ms/step - loss: 161.8945 - val_loss: 98.4162
Epoch 2/100
124/124 [==============================] - 3s 25ms/step - loss: 165.9593 - val_loss: 98.2429
Epoch 3/100
124/124 [==============================] - 3s 25ms/step - loss: 159.7672 - val_loss: 97.3229
Epoch 4/100
124/124 [==============================] - 3s 25ms/step - loss: 160.4258 - val_loss: 98.8134
Epoch 5/100
124/124 [==============================] - 3s 26ms/step - loss: 161.9977 - val_loss: 99.6885
Epoch 6/100
124/124 [==============================] - 3s 25ms/step - loss: 163.6254 - val_loss: 99.2342
Epoch 7/100
124/124 [==============================] - 3s 27ms/step - loss: 159.2067 - val_loss: 97.8587
Epoch 8/100
124/124 [==============================] - 4s 28ms/step - loss: 155.5349 - val_loss: 97.8207
Epoch 9/100
124/124 [==============================] - 4s 32ms/step - loss: 162.9657 - val_loss: 97.6615
Epoch 10/100
124/124 [==============================] -

In [16]:

# Save the model
model.save('Morrisons_lstm_model1.h5')


# Load the model
model = load_model('Morrisons_lstm_model1.h5',compile=False)


import pickle

# Save label encoders
with open('Morrisons_label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)


# Load label encoders
with open('Morrisons_label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)



# model.predict(test_X)


In [26]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import pickle
from sklearn.preprocessing import LabelEncoder

# Load the demand dataset
df_demand = pd.read_csv('Demand_Dataset1.csv')

# Load the model and label encoders
model = load_model('Morrisons_lstm_model1.h5', compile=False)

with open('Morrisons_label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

# Function to prepare the latest sequence for prediction
def prepare_latest_sequence(df, product_name, sequence_length, label_encoders):
    df_product = df[df['names'] == product_name]
    df_product = df_product.sort_values(by=['year', 'month', 'day'])
    latest_sequence = df_product.tail(sequence_length)
    
    if len(latest_sequence) < sequence_length:
        padding = sequence_length - len(latest_sequence)
        latest_sequence = pd.concat([df_product] * (padding + 1), ignore_index=True).tail(sequence_length)
    
    # Encode categorical features
    for column, le in label_encoders.items():
        if column in latest_sequence.columns:
            if set(latest_sequence[column].unique()).difference(set(le.classes_)):
                # Handle unseen labels
                new_classes = list(set(latest_sequence[column].unique()).difference(set(le.classes_)))
                le.classes_ = np.append(le.classes_, new_classes)
            latest_sequence[column] = le.transform(latest_sequence[column])
    
    # Ensure all columns are present
    all_features = list(label_encoders.keys()) + ['unit_price']
    for feature in all_features:
        if feature not in latest_sequence.columns:
            latest_sequence[feature] = 0  # Fill missing columns with zeros
    
    # Reorder columns to match model input
    latest_sequence = latest_sequence[all_features]
    

    X_latest = latest_sequence.drop(columns=['unit_price']).select_dtypes(include=[np.number])
    
    # Pad feature columns if there are fewer than expected features
    expected_features = 11
    if X_latest.shape[1] < expected_features:
        padding = expected_features - X_latest.shape[1]
        X_latest = np.pad(X_latest.values, ((0, 0), (0, padding)), mode='constant')
    elif X_latest.shape[1] > expected_features:
        X_latest = X_latest.iloc[:, :expected_features]
    
    return X_latest

# Function to predict unit price for a given number of days
def predict_prices(model, product_name, label_encoders, sequence_length, num_days):
    predictions = []
    latest_sequence_df = prepare_latest_sequence(df, product_name, sequence_length, label_encoders)
    
    # Ensure data is in correct shape and type
    X_latest = latest_sequence_df.reshape(1, sequence_length, latest_sequence_df.shape[1])
    X_latest = X_latest.astype(np.float32)
    
    for _ in range(num_days):
        next_day_price = model.predict(X_latest)
        predictions.append(next_day_price[0][0])
        
        new_sequence = np.zeros((sequence_length, X_latest.shape[2]))
        new_sequence[0:-1] = X_latest[0, 1:, :]
        new_sequence[-1] = np.append(X_latest[0, -1, :-1], next_day_price[0][0])
        X_latest = new_sequence.reshape(1, sequence_length, X_latest.shape[2])
    
    return predictions

# Function to calculate dynamic price
def calculate_dynamic_price(predicted_price, df_demand, product_name, weekday):
    product_demand = df_demand[(df_demand['product'] == product_name) & (df_demand['weekday'] == weekday)]
    asda_sales = product_demand[product_demand['supermarket'] == 'Morrisons']['total_sale'].sum()
    competitor_sales = product_demand[product_demand['supermarket'] != 'Morrisons'].groupby('supermarket')['total_sale'].sum().sum()
    
    if asda_sales > 0:
        price_adjustment_factor = competitor_sales / (asda_sales + 1)
        dynamic_price = predicted_price * (1 + price_adjustment_factor)
    else:
        dynamic_price = predicted_price
    
    return dynamic_price

# Example usage
product_name = 'Pukka Minced Beef & Onion Pie'
sequence_length = 10
num_days = 1
weekday = 5  # Example: Tuesday

predicted_prices = predict_prices(model, product_name, label_encoders, sequence_length, num_days)
dynamic_prices = [calculate_dynamic_price(price, df_demand, product_name, weekday) for price in predicted_prices]

print(f"Predicted Unit Prices: {predicted_prices}")
print(f"Dynamic Prices: {dynamic_prices}")


c:\Users\ADMIN\anaconda3\envs\ml_env\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Predicted Unit Prices: [3.0624936]
Dynamic Prices: [12.249974250793457]
